In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
#cd gdrive/My Drive/

/content/gdrive/My Drive


In [ ]:
#!zip -r Dataset.zip colbert-trained/
#!unzip filename.zip -d ./filename

  adding: colbert-trained/ (stored 0%)
  adding: colbert-trained/saved_model.pb (deflated 92%)
  adding: colbert-trained/variables/ (stored 0%)
  adding: colbert-trained/variables/variables.index (deflated 80%)
  adding: colbert-trained/variables/variables.data-00000-of-00001 (deflated 14%)


In [ ]:
#Saving the image features
'''
np.save('trainimage_features.npy', trainImageFeatures)
np.save('valimage_features.npy', valImageFeatures)
np.save('testimage_features.npy', testImageFeatures)

from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

trainImageFeatures = np.load('/content/gdrive/MyDrive/trainimage_features.npy') 
valImageFeatures = np.load('/content/gdrive/MyDrive/valimage_features.npy')
testImageFeatures = np.load('/content/gdrive/MyDrive/testimage_features.npy')
'''

"\nnp.save('trainimage_features.npy', trainImageFeatures)\nnp.save('valimage_features.npy', valImageFeatures)\nnp.save('testimage_features.npy', testImageFeatures)\n\nfrom google.colab import drive\ndrive.mount('/content/gdrive',force_remount=True)\n\ntrainImageFeatures = np.load('/content/gdrive/MyDrive/trainimage_features.npy') \nvalImageFeatures = np.load('/content/gdrive/MyDrive/valimage_features.npy')\ntestImageFeatures = np.load('/content/gdrive/MyDrive/testimage_features.npy')\n"

In [ ]:
import keras

model = keras.models.load_model("/content/gdrive/MyDrive/colbert-trained/")
model.summary()


Model: "functional_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                       

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
#from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

2.8.0


In [ ]:
training_sample_count = 1000 # 4000
test_count = 1000

MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [ ]:
df = pd.read_csv('cyberbullying_tweets.csv')

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df, test_size = 0.20, shuffle = True)

In [ ]:
df_train.head(5)

,tweet_text,label
38119,"I’m WEAK!!! Not even close, but go off sis you...",4
35602,Me in high school getting bullied by all the m...,4
9579,RT @Shimmyfab: @Icarusflyer @JustLaurenB @wesl...,1
29669,Let's not forget that wadhwa is the same perso...,3
18865,Idiot! He was not in power at that time. BJP w...,2


In [ ]:
def smush_labels(label):
    if label == 'not_cyberbullying':
        return 0
    elif label == 'gender':
        return 1
    elif label == 'religion':
        return 2
    elif label == 'other_cyberbullying':
        return 3
    elif label == 'age':
        return 4
    else: #ethnicity
        return 5

df_train['label'] = df_train['cyberbullying_type'].apply(smush_labels)
df_test['label'] = df_test['cyberbullying_type'].apply(smush_labels)

In [ ]:
del df_test['cyberbullying_type']
del df_train['cyberbullying_type']

In [ ]:
!pip install emoji

     |████████████████████████████████| 175 kB 15.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=ccd9eac2f02e87fdea5a08ac34340d2ae2cc2d22b7ac339b607244a7783f2020
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Text cleaning
import re, string
import emoji
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
#Clean emojis from text
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji

#Remove punctuations, links, stopwords, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    #banned_list= string.punctuation
    #table = str.maketrans('', '', banned_list)
    #text = text.translate(table)
    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)
    text =' '.join(word for word in text.split() if len(word) < 14) # remove words longer than 14 characters
    return text

#remove contractions
def decontract(text):
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the "#" symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as "&" and "$" present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

#Remove multiple sequential spaces
def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)

#Stemming
def stemmer(text):
    tokenized = nltk.word_tokenize(text)
    ps = PorterStemmer()
    return ' '.join([ps.stem(words) for words in tokenized])

#Lemmatization 
#NOTE:Stemming seems to work better for this dataset
def lemmatize(text):
    tokenized = nltk.word_tokenize(text)
    lm = WordNetLemmatizer()
    return ' '.join([lm.lemmatize(words) for words in tokenized])

#Then we apply all the defined functions in the following order
def deep_clean(text):
    text = strip_emoji(text)
    text = decontract(text)
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    text = stemmer(text)#check this
    return text

In [ ]:
df_train['tweet_text'] = df_train['tweet_text'].apply(deep_clean)
df_test['tweet_text'] = df_test['tweet_text'].apply(deep_clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  if sys.path[0] == '':


In [ ]:
df_train.head(5)

,tweet_text,label
38119,im weak even close go si bulli boyfriend that ...,4
35602,high school get bulli mean girl shi introvert ...,4
9579,rt wont drive 2 stop rape class http,1
29669,let forget wadhwa person believ pseudosci need...,3
18865,idiot power time bjp power vp singh modi power...,2


In [ ]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['tweet_text']

output TARGET_COUNT:
	 1

output categories:
	 ['label']


In [ ]:
from transformers import BertTokenizer

MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = sent_tokenize(row.tweet_text)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.tweet_text, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input

In [ ]:
inputs      = compute_input_arrays(df_train, input_categories, tokenizer)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer)

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(38153, 20)


0it [00:00, ?it/s]

(9539, 20)


In [ ]:
print(len(inputs), len(inputs[0]), len(inputs[0][0]))

# check out input for 7th row
xx = 7
#print(df_train.iloc[xx,0])
#print(sent_tokenize(df_train.iloc[xx,0]))
#inputs[0][xx], inputs[3][xx], inputs[6][xx], inputs[15][xx]
inputs[0][200]

18 38153 20


array([  101,  9033,  6894,  8257,  1040, 12273,  6102,  5607,  2082,
       21146,  4360,  2375,  3499,  9040,  2775,  8273,  2015,  5466,
        5637,   102], dtype=int32)

In [ ]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(df_train, output_categories)
outputs[:10]

array([[4],
       [4],
       [1],
       [3],
       [2],
       [1],
       [0],
       [3],
       [0],
       [5]])

In [ ]:
valid_inputs = inputs
valid_outputs = outputs

In [ ]:
from keras.models import Model
model2= Model(inputs=model.input, outputs=model.layers[-2].output)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, GRU, RNN, SimpleRNN, Dropout

In [ ]:
fc = Dense(6,activation='softmax')(model2.output)

check_model = Model(inputs=model2.input, outputs=fc)

In [ ]:
check_model.summary()
#https://stackoverflow.com/questions/45755022/cannot-add-layers-to-saved-keras-model-model-object-has-no-attribute-add

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                            

In [ ]:
for layer in check_model.layers[:-1]:
        layer.trainable = False

In [ ]:
check_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
test_outputs = compute_output_arrays(df_test, output_categories)
test_outputs[:2]

array([[0],
       [3]])

In [ ]:
#from sklearn.preprocessing import OneHotEncoder

#onehot_encoder = OneHotEncoder(sparse=False)
#test_outputs_encoded = onehot_encoder.fit_transform(test_outputs)

#valid_outputs_encoded = onehot_encoder.fit_transform(valid_outputs)


In [ ]:
test_outputs_encoded = tf.keras.utils.to_categorical(test_outputs, num_classes=6)
valid_outputs_encoded = tf.keras.utils.to_categorical(valid_outputs, num_classes=6)

In [ ]:
#tf.config.run_functions_eagerly(False)

In [ ]:
len(test_outputs_encoded)

9539

In [ ]:
check_model.layers[-2].trainable

False

In [ ]:
check_model.fit(valid_inputs, valid_outputs_encoded,
          batch_size=32,
          epochs=5,  # only 2 epochs, for demonstration purposes
          verbose=1,
          validation_data=(test_inputs, test_outputs_encoded))
#
          #steps_per_epoch=100, validation_steps=10,
          #validation_data=(test_inputs, test_outputs_encoded)

Epoch 1/5
   6/1193 [..............................] - ETA: 8:29 - loss: 1.7291 - accuracy: 0.1771

KeyboardInterrupt: ignored

In [ ]:
test_preds = check_model.predict(test_inputs)

In [ ]:
test_loss, test_acc = check_model.evaluate(test_inputs,  test_outputs_encoded, verbose=2)

print('\nTest accuracy:', test_acc)

299/299 - 122s - loss: 1.7300 - accuracy: 0.2313 - 122s/epoch - 409ms/step

Test accuracy: 0.23126113414764404


In [ ]:
test_preds

array([[0.12283842, 0.1255345 , 0.22886367, 0.10736358, 0.18325388,
        0.23214601],
       [0.13627303, 0.11630222, 0.21561876, 0.11592054, 0.18740356,
        0.22848187],
       [0.23645478, 0.15791282, 0.14679839, 0.12363876, 0.23502466,
        0.1001706 ],
       ...,
       [0.12959604, 0.12357841, 0.2253802 , 0.09921533, 0.1690061 ,
        0.25322393],
       [0.13471094, 0.12331535, 0.22231829, 0.13621874, 0.17472155,
        0.20871508],
       [0.18075736, 0.16516162, 0.20811665, 0.11500063, 0.18757764,
        0.14338608]], dtype=float32)

In [ ]:
check_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          multiple             0           []                               
                                                                                                  
 input_21 (InputLayer)          multiple             0           []                               
                                                                                                  
 input_22 (InputLayer)          multiple             0           []                               
                                                                                            